In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import os
import smtplib
import email.message
from dotenv import load_dotenv
from pathlib import Path 
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [2]:
driver=webdriver.Chrome()
url = 'https://skiplagged.com/flights/FLN/madrid/2024-04-06'
driver.get(url)
sleep(20)

The chromedriver version (117.0.5938.92) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (118.0.5993.70); currently, chromedriver 118.0.5993.70 is recommended for chrome 118.*, so it is advised to delete the driver in PATH and retry


In [3]:
content = driver.page_source
soup = BeautifulSoup(content)

In [4]:
for span in soup.findAll('span',attrs={'class':'span2 trip-cost'}):
    price = span.text[-4:]

In [5]:
print(soup)

<html lang="pt"><head>
<script async="" src="https://js.appboycdn.com/web-sdk/4.1/braze.min.js" type="text/javascript"></script><script async="" src="https://connect.facebook.net/en_US/fbevents.js" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtm.js?id=GTM-P483ZNT"></script><script defer="" id="growthbook-sdk" src="https://cdn.jsdelivr.net/npm/@growthbook/growthbook/dist/bundles/index.min.js"></script>
<script async="" src="https://www.googletagmanager.com/gtag/js?id=G-LKWPZ145DS"></script>
<script>
  (function() {
    window.dataLayer = window.dataLayer || [];
    function gtag(){window.dataLayer.push(arguments);}
    gtag('js', new Date());
    gtag('config', 'G-LKWPZ145DS');

    // Wait for the SDK to load before starting GrowthBook
    if (window.growthbook) {
      startGrowthbook();
    } else {
      document.querySelector("#growthbook-sdk").addEventListener("load", startGrowthbook);
    }

    function startGrowthbook() {
      if (!wi

In [6]:
temp_price = soup.find("div", {"class": "span2 trip-cost"})
price = temp_price.find("span", {"class": ""})
lst_price = []
lst_price.append(price.text)

In [7]:
print(lst_price)

['R$\xa02.560']


In [8]:
lista_limpa = lst_price
lista_limpa = [w.replace('R$\xa0', '') for w in lista_limpa]
lista_limpa = [w.replace('.', '') for w in lista_limpa]
menor_preco = lista_limpa[0]
print(menor_preco)

2560


In [9]:
def enviar_email():
    msg = MIMEMultipart()
            
    message = "Voo por R$ "+str(menor_preco)+ "\n\n no link "+str(url);

    env_path = Path('.')/'.env'
    load_dotenv(dotenv_path = env_path)
            
    #msg = email.message.Message()
    msg['Subject'] = "Voo abaixo de 2k"
    msg['From'] = 'Aqui vai o email remetente'
    msg['To'] = 'Aqui vai email destino'
    password = 'Aqui vai a senha' 
    msg.add_header('Content-Type', 'text/html')
    #msg.set_payload(html )
            
    # add in the message body
    msg.attach(MIMEText(message, 'html'))

    s = smtplib.SMTP('smtp.gmail.com: 587')
    s.starttls()
    # Login Credentials for sending the mail
    s.login(msg['From'], password)
    s.sendmail(msg['From'], [msg['To']], msg.as_string())
    s.quit()

In [10]:
if int(menor_preco)<=4000:
    enviar_email()
    print("Email Enviado")
#driver.quit()
#sleep(50)

Email Enviado
